In [ ]:
# Imports and API Key Setup
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load API keys from your .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LLM_MODEL = "gpt-4o" # Use the same model as your RAG pipeline for a fair comparison

if not OPENAI_API_KEY:
    print("Error: OPENAI_API_KEY not found. Please set it in your .env file.")
else:
    print(f"OpenAI API Key loaded. Using model: {LLM_MODEL}")

In [ ]:
# Define the "Benchmark" Chain (No RAG)

# 1. Initialize the LLM (same as your RAG pipeline)
llm = ChatOpenAI(
    model=LLM_MODEL,
    api_key=OPENAI_API_KEY
)

# 2. Define the Prompt Template
# This prompt asks the LLM to answer from its own knowledge,
# unlike the RAG prompt which FORCED it to use provided context.
prompt_template_str = """You are a helpful financial analyst assistant.

Answer the following question regarding 10q reports in SEC Fillings to the best of your ability based on your general, pre-trained knowledge.

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(prompt_template_str)

# 3. Define a simple string output parser
output_parser = StrOutputParser()

# 4. Create the "benchmark" chain
# This is the "base benchmark" chain: Prompt -> LLM -> Output
benchmark_chain = prompt | llm | output_parser

print("Benchmark (non-RAG) chain created successfully.")

In [ ]:
# Run Queries and Get Answers
# TO BE CHANGED AFTER EVALUATION QA ARE GENERATED
# Use the same questions from your 'base_RAG.ipynb' for a fair comparison
questions = [
    "What are the main risk factors mentioned by tech companies?",
    "What risks did Apple disclose in their latest 10-Q?",
    "Compare the revenue trends of NVIDIA and AMD",
    "What was Tesla's R&D spending in the latest quarter?"
]

print(f"--- STARTING BENCHMARK QUERIES (NO RAG) ---")

for q in questions:
    print(f"\n{'='*70}")
    print(f"QUERY: {q}")
    print(f"{'-'*70}")

    # Invoke the chain
    answer = benchmark_chain.invoke({"question": q})

    print(f"ANSWER (from gpt-4o's internal knowledge):\n{answer}")
    print(f"{'='*70}")

print("\n--- BENCHMARK QUERIES COMPLETE ---")